In [1]:
from brainlit.utils import session
import napari

In [2]:
url = "s3://mouse-light-viz/precomputed_volumes/brain1"
sess = session.NeuroglancerSession(url=url, url_segments=url+"_segments", mip=0)

In [3]:
img, _, vert = sess.pull_chunk(2, 0)

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]


In [4]:
with napari.gui_qt():
    viewer = napari.view_image(img, ndisplay=3)